# Plot major velocity cluster boundaries and selected domains
The cluster boundaries are picked manually based on votes >=3. This script plots the picked boundaries and domains, overlaying on the cluster boundary votes.

In [6]:
import pickle,os
import matplotlib.pyplot as plt
import numpy as np
from seisgo import utils
# from scipy import ndimage
from matplotlib.colors import ListedColormap
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rc('font',family='Helvetica')

In [7]:
tagbase="euclidean"
datadir="kresults_"+tagbase+"_smooth_resampled"
figdir="plots_"+tagbase+"_smooth_resampled"

cmap=plt.cm.turbo
f_file="../faults_terranes/N_Cordillera_major_faults_2020.gmt"
t_file="../faults_terranes/N_Cordillera_terranes_2020.gmt"

faults,ftags=utils.read_gmtlines(f_file)
terranes,ttags=utils.read_gmtlines(t_file)
##Latitude
minlat=55
maxlat=71.5

##Longitude
minlon=-168
maxlon=-128
proj=ccrs.LambertAzimuthalEqualArea(central_longitude=np.mean([minlon,maxlon]), 
                                  central_latitude=np.mean([minlat,maxlat]), 
                                  false_easting=0.0, false_northing=0.0,globe=None)
with open('edges_allmodels_samegrid.pk','rb') as f:
    edgesall_samegrid=pickle.load(f)
dirall=list(edgesall_samegrid.keys())

### Cutoff on votes and plot as constant color

In [8]:
savefig=True
maxn=len(edgesall_samegrid[dirall[0]]['data'])#np.max(gradall)
minn=3
for i,d in enumerate(dirall):
# if 1:
#     i=1
#     d=dirall[i]
    if i == 0:
        outlinefile="velocity_domain_outlines_crust.txt"
        boundaryfile="cluster_boundary_minvote3_crust.txt"
    elif i ==1:
        outlinefile="velocity_domain_outlines_mantle.txt"
        boundaryfile="cluster_boundary_minvote3_mantle.txt"
    boundaries,btags=utils.read_gmtlines(boundaryfile)
    outlines,otags=utils.read_gmtlines(outlinefile)
    print(d)
    edges=edgesall_samegrid[d]
    vecx=edges['x']
    vecy=edges['y']
    gradall=np.zeros((len(vecx),len(vecy)))
    for e in range(len(edges['data'])):
        grad=edges['data'][e]
        gradall += grad

    #
    gradalltemp=gradall.copy()
    gradalltemp[gradalltemp<minn]=np.nan
    plt.figure(facecolor='w')
    ax = plt.axes(projection=proj)
    plt.pcolor(vecx,vecy,gradalltemp.T,cmap=ListedColormap(['blue']),shading='auto',
            transform=ccrs.PlateCarree())
    tcount=0
    for k in range(len(ttags)):
        if len(terranes[k][:,0]) > 400:
            if tcount==0:
                plt.fill(terranes[k][:,0],terranes[k][:,1],'m',facecolor='none',
                    edgecolor='c',lw=0.75,alpha=1,ls=':',transform=ccrs.PlateCarree(),
                    label="terranes")
            else:
                plt.fill(terranes[k][:,0],terranes[k][:,1],'m',facecolor='none',
                edgecolor='c',lw=0.75,alpha=1,ls=':',transform=ccrs.PlateCarree())
            tcount += 1
    for k in range(len(ftags)):
        if k==0:
            plt.plot(faults[k][:,0],faults[k][:,1],'-',color='g',
                    lw=0.75,alpha=1,transform=ccrs.PlateCarree(),label='faults')
        else:
            plt.plot(faults[k][:,0],faults[k][:,1],'-',color='g',
                    lw=0.75,alpha=1,transform=ccrs.PlateCarree())
    for k in range(len(btags)):
        if k==0:
            plt.plot(boundaries[k][:,0],boundaries[k][:,1],'r--',lw=2,transform=ccrs.PlateCarree(),
                        label="domains")
        else:
            plt.plot(boundaries[k][:,0],boundaries[k][:,1],'r--',lw=2,transform=ccrs.PlateCarree())
    for k in range(len(otags)):
        xshift=0
        yshift=0
        if i==0:
            domaintag='C'
            if otags[k] == 'outline-2' or otags[k] == 'outline-3':
                xshift=1
                yshift=-0.5
            elif otags[k] == 'outline-4':
                xshift=-1
                yshift=-1.5 
            elif otags[k] == 'outline-5':
                xshift=-1.5
                yshift=0 
            elif otags[k] == 'outline-6':
                xshift=0
                yshift=0.5 
        elif i==1:
            domaintag='M'
            if otags[k] == 'outline-1':
                xshift=-2.5
                yshift=-0.5
            elif otags[k] == 'outline-2':
                xshift=0
                yshift=-1.7
            elif otags[k] == 'outline-3':
                xshift=0
                yshift=0.4
            elif otags[k] == 'outline-4':
                xshift=0
                yshift=0.6
            elif otags[k] == 'outline-5':
                xshift=-3.5
                yshift=0 
            elif otags[k] == 'outline-6':
                xshift=-0.5
                yshift=0
        plt.text(xshift+np.mean(outlines[k][:,0]),yshift+np.mean(outlines[k][:,1]),
                domaintag+otags[k].replace('outline-',''),fontsize=10,color='r',
                transform=ccrs.PlateCarree())
    ax.set_extent((minlon,maxlon,minlat*0.95,maxlat*1.0))
    ax.coastlines(resolution='10m',color='k', linewidth=.5)
    gl=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True, dms=True, x_inline=False, y_inline=False,
                    color='gray', alpha=0.5, linestyle='--',linewidth=0.5)
    gl.xlocator = mticker.FixedLocator(np.arange(minlon,maxlon,5))
    gl.ylocator = mticker.FixedLocator(np.arange(minlat,maxlat,2))
    plt.title('minimum votes: '+str(minn))
    plt.legend(fontsize=10,loc='upper right')
    if savefig:
        figname=os.path.split(d)[-1]+"_clustermap_domains_cut"+str(minn)+".pdf"
        plt.savefig(figname,format='pdf',dpi=300)
        plt.close()
    else:
        plt.show()
    # clean without faults and terranes
    plt.figure(facecolor='w')
    ax = plt.axes(projection=proj)
    plt.pcolor(vecx,vecy,gradalltemp.T,cmap=ListedColormap(['blue']),shading='auto',
            transform=ccrs.PlateCarree())
    for k in range(len(btags)):
        if k==0:
            plt.plot(boundaries[k][:,0],boundaries[k][:,1],'r--',lw=2,transform=ccrs.PlateCarree(),
                        label="domains")
        else:
            plt.plot(boundaries[k][:,0],boundaries[k][:,1],'r--',lw=2,transform=ccrs.PlateCarree())
    for k in range(len(otags)):
        xshift=0
        yshift=0
        if i==0:
            domaintag='C'
            if otags[k] == 'outline-2' or otags[k] == 'outline-3':
                xshift=1
                yshift=-0.5
            elif otags[k] == 'outline-4':
                xshift=-1
                yshift=-1.5 
            elif otags[k] == 'outline-5':
                xshift=-1.5
                yshift=0 
            elif otags[k] == 'outline-6':
                xshift=0
                yshift=0.5 
        elif i==1:
            domaintag='M'
            if otags[k] == 'outline-1':
                xshift=-2.5
                yshift=-0.5
            elif otags[k] == 'outline-2':
                xshift=0
                yshift=-1.7
            elif otags[k] == 'outline-3':
                xshift=0
                yshift=0.4
            elif otags[k] == 'outline-4':
                xshift=0
                yshift=0.6
            elif otags[k] == 'outline-5':
                xshift=-3.5
                yshift=0 
            elif otags[k] == 'outline-6':
                xshift=-0.5
                yshift=0
        plt.text(xshift+np.mean(outlines[k][:,0]),yshift+np.mean(outlines[k][:,1]),
                domaintag+otags[k].replace('outline-',''),fontsize=10,color='r',
                transform=ccrs.PlateCarree())
    ax.set_extent((minlon,maxlon,minlat*0.95,maxlat*1.0))
    ax.coastlines(resolution='10m',color='k', linewidth=.5)
    gl=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True, dms=True, x_inline=False, y_inline=False,
                    color='gray', alpha=0.5, linestyle='--',linewidth=0.5)
    gl.xlocator = mticker.FixedLocator(np.arange(minlon,maxlon,5))
    gl.ylocator = mticker.FixedLocator(np.arange(minlat,maxlat,2))
    plt.title('minimum votes: '+str(minn))
    plt.legend(fontsize=10,loc='upper right')
    if savefig:
        figname=os.path.split(d)[-1]+"_clustermap_domains_cut"+str(minn)+"_clean.pdf"
        plt.savefig(figname,format='pdf',dpi=300)
        plt.close()
    else:
        plt.show()


kresults_euclidean_smooth_resampled/10_50km


1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped


kresults_euclidean_smooth_resampled/40_120km


1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
